In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np
import datetime as dt
import time
import json
from pyspark.sql import SQLContext
import json
from IPython import display
from IPython.core.display import HTML
import pandas
import time 
import numpy as np
import matplotlib.pyplot as plt 
from pyspark.sql.functions import split, explode, col

sql = SQLContext(sc)

#  1ST prediction :NEW CASTLE HABOUR PROTEST (8TH MARCH 2016)
# Event Description:

On 8th March 2016 about 1,500 anti-coal protesters descended on the NSW port of Newcastle on Sunday in a bid to blockade the port and prevent exports. The world's largest coal export port, north of Sydney, was targeted by demonstrators calling for action to tackle climate change. About noon, the port was filled with kayaks paddling out to the entrance of the harbour as part of the Break Free event. The aim was for the mass flotilla to stop coal ships entering and leaving the harbour, something protesters claimed to have achieved. Police said 66 people were arrested during the "large civil disobedience" protest. In total, 57 protesters who occupied a rail bridge to block coal train at Sandgate were issued with Field Court Attendance Notices (FCANs) for remaining on enclosed lands. Pictures posted on social media showed groups of protesters lying on train tracks. The day of protest was organized by a range of groups, supported by Greenpeace and climate action group 350.org. Hundreds of Rising Tide protesters took to Newcastle Harbour for its seventh annual anti-coal port blockade. The protesters took to canoes and kayaks to block the port and deny coal ships entry for the majority of the day, the Newcastle Herald reports. 
The popular hashtags for the event were:

#breakfree    #keepitintheground    #ClimateAction     #Newcastle   #endcoal     #ClimateResistance  #quitcoal          #coal       #endfossilfuel 

# Get one Week Tweet Before the Event....

In [71]:
lol = sql.read.load(format = "au.com.d2dcrc.carbon.spark.tweets", startTime = "2016-05-07T12:00:00Z", endTime = "2016-05-07T23:29:59Z")
lol.printSchema()


displaytweets = lol.select( "author", "publicationTime", "bodyText", "locations").orderBy("publicationTime")
#pd.set_option('display.max_colwidth', 500)
#display.display(displaytweets.toPandas())

root
 |-- key:salt: byte (nullable = true)
 |-- key:timestamp: timestamp (nullable = true)
 |-- key:id: string (nullable = true)
 |-- key:subIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- data: string (nullable = true)
 |-- reference: string (nullable = true)
 |-- format: string (nullable = true)
 |-- type: string (nullable = true)
 |-- source: string (nullable = true)
 |-- ingestTime: timestamp (nullable = true)
 |-- author: string (nullable = true)
 |-- civilEvents: string (nullable = true)
 |-- locations: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- nlp: string (nullable = true)
 |-- publicationTime: timestamp (nullable = true)
 |-- politicalParty: string (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- times: string (nullable = true)
 |-- bodyText: string (nullable = true)
 |-- title: string (nullable = true)
 |-- phraseList: string (nullable = true)
 |-- spam: string (nullable = true)



In [72]:
import time 
startTime = time.time()
sqlContext.registerDataFrameAsTable(displaytweets, "doall")
saveall = sqlContext.sql("SELECT * FROM doall")
saveall.describe()
sly1 = saveall.toPandas()
#sly1.to_csv('realFinalNHB.csv')
print('The script took {0} second !'.format(time.time() - startTime))

The script took 91.24707555770874 second !


In [61]:
#print(len(sly1))
#sly1.head()

In [74]:
#keywords = ['#coal', '#shenhua', '#csg', '#keepintheground', '#breakfree', '#coal', 'liverpoolplains']


keywords = ["protest", "muslim", "islam", "outcry", "asylum", "organisation", "threat", "union", "centrelink",
            "opposition", "parade", "council",
            "federal", "strike", "harass", "refugee", "riot", "community", "reclaim", "poster", "demonstration",
            "petition", "funding", "barrier", "march",
            "crowd", "celebration", "action", "barricade", "placard", "gather", "resident", "patriot", "bigot",
            "racism", "national", "decision", "movement",
            "mentality", "racist", "agency", "mosque", "highlight", "halaal", "turmoil", "activist", "disturbance",
            "victory", "equality", "blockade", "anger",
"ideal", "unite", "extremist", "anzac", "rally", "culture", "unrest", "terror", "terrorist"
, "prevent", "cpsu", "anger", "concern","stoppage", "park","value", "claim","immigrant","opponent",
                 "standoff", "spray","sign","urban","pressure","destroy","side", "unrest", "stoking"]

In [ ]:
import pandas as pd
import datetime as dt
import multiprocessing as mp


def matches_util(args):
    df_in, word_list, min_keywords = args

    # Check for every keyword in ever row
    values = pd.DataFrame()
    for kw in word_list:
        values[kw] = df_in.bodyText.str.contains(kw, case=False)

    # Add keyword counts
    values["wordCnt"] = [v.sum() for k, v in values.iterrows()]

    # If keyword count is greater than min_keywords then include in result
    return df_in.loc[values.wordCnt >= min_keywords]


def get_matches(df_in, word_list, min_keywords=2, output_filepath="", multi_threads=False, thread_cnt=2):
    print("Looking for matches.\nmin_keywords={}\noutput_filepath={}\nmulti_threads={}\nword_list={}".format(
        min_keywords,
        output_filepath,
        multi_threads,
        word_list))

    # Make sure bodyText field exists
    assert "bodyText" in df_in.columns, "Field 'bodyText' was not found in input DataFrame!"
    ts1 = dt.datetime.now()
    result = None
    resList = None

    # Use multi-threading if enabled
    if multi_threads:
        with mp.Pool(thread_cnt) as p:
            threadCnt = thread_cnt
            bins = pd.cut(df_in.index, threadCnt, labels=range(threadCnt))
            resList = p.map(matches_util, ((df_in.loc[bins == i], keywords, min_keywords) for i in range(thread_cnt)))

    # Else process without multi-threading
    else:
        result = matches_util((df_in, keywords, min_keywords))

    # Get results from queue
    if result is None:
        result = pd.concat(resList)

    ft = dt.datetime.now() - ts1
    print("Finished looking for matches.  Time Elapsed={:.1f}s".format(ft.total_seconds()))

    # Save resulting matches to file if a filepath was passed
    if output_filepath != "":
        result.to_csv(output_filepath)
        print("Saved output to {}".format(output_filepath))

    return result

In [ ]:
get_matches(sly1, word_list=keywords, min_keywords=2, output_filepath="output3.csv", multi_threads=True, thread_cnt=8)

In [ ]:
import time 
startTime = time.time()
aa = get_matches2(sly1, keywords, 2, "output3.csv")
#print(aa)
print('The script took {0} second !'.format(time.time() - startTime))
import time 

startTime = time.time()
sqlContext.registerDataFrameAsTable(displaytweets, "timePlus")
feb10 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-07%'")
f10 = feb10.toPandas()

f10.to_csv('short.csv')
f10.describe()
print('The script took {0} second !'.format(time.time() - startTime))
#hashTags = newpd.where(newpd['bodyText'].str.contains('#coal')).dropna() #this works thesame as

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb11 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-11%'")
f11 = feb11.toPandas()
f11.to_csv('Feb11.csv')
f11.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb12 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-12%'")
f12 = feb12.toPandas()
f12.to_csv('Feb12.csv')
f12.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb13 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-13%'")
f13 = feb13.toPandas()
f13.to_csv('Feb13.csv')
f13.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb14 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-14%'")
f14 = feb14.toPandas()
f14.to_csv('Feb14.csv')
f14.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb15 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-15%'")
f15 = feb15.toPandas()
f15.to_csv('Feb15.csv')
f15.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb16 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-16%'")
f16 = feb16.toPandas()
f16.to_csv('Feb16.csv')
f16.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb17 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-17%'")
f17 = feb17.toPandas()
f17.to_csv('Feb17.csv')
f17.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb18 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-18%'")
f18 = feb18.toPandas()
f18.to_csv('Feb17.csv')
f18.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb19 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-19%'")
f19 = feb19.toPandas()
f19.to_csv('Feb19.csv')
f19.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb20 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-20%'")
f20 = feb20.toPandas()
f20.to_csv('Feb20.csv')
f20.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb21 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-21%'")
f21 = feb21.toPandas()
f21.to_csv('Feb21.csv')
f21.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb22 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-22%'")
f22 = feb22.toPandas()
f22.to_csv('Feb22.csv')
f22.describe()
#rundata again and change name

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb23 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-23%'")
f23 = feb23.toPandas()
f23.to_csv('Feb22.csv')
f23.describe()
#run data again and change the name

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb23 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-23%'")
f23 = feb23.toPandas()
f23.to_csv('Feb23.csv')
f23.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb24 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-24%'")
f24 = feb24.toPandas()
f24.to_csv('Feb24.csv')
f24.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb24 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-24%'")
f24 = feb24.toPandas()
f24.to_csv('Feb24.csv')
f24.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb25 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-25%'")
f25 = feb25.toPandas()
f25.to_csv('Feb25.csv')
f25.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb26 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-26%'")
f26 = feb26.toPandas()
f26.to_csv('Feb26.csv')
f26.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb27 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-27%'")
f27 = feb27.toPandas()
f27.to_csv('Feb27.csv')
f27.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb28 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-28%'")
f28 = feb28.toPandas()
f28.to_csv('Feb28.csv')
f28.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
feb29 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-02-29%'")
f29 = feb29.toPandas()
f29.to_csv('Feb29.csv')
f29.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar1 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-01%'")
m1 = Mar1.toPandas()
m1.to_csv('Mar1.csv')
m1.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar2 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-02%'")
m2 = Mar2.toPandas()
m2.to_csv('Mar2.csv')
m2.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar3 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-03%'")
m3 = Mar3.toPandas()
m3.to_csv('Mar3.csv')
m3.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar4 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-04%'")
m4 = Mar4.toPandas()
m4.to_csv('Mar4.csv')
m4.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar5 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-05%'")
m5 = Mar5.toPandas()
m5.to_csv('Mar5.csv')
m5.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar6 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-06%'")
m6 = Mar6.toPandas()
m6.to_csv('Mar6.csv')
m6.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar7 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-07%'")
m7 = Mar7.toPandas()
m7.to_csv('Mar7.csv')
m7.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar8 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-08%'")
m8 = Mar8.toPandas()
m8.to_csv('Mar8.csv')
m8.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar9 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-09%'")
m9 = Mar9.toPandas()
m9.to_csv('Mar9.csv')
m9.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar10 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-10%'")
m10 = Mar10.toPandas()
m10.to_csv('Mar10.csv')
m10.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar11 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-11%'")
m11 = Mar11.toPandas()
m11.to_csv('Mar11.csv')
m11.describe()
#rerun

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar12 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-12%'")
m12 = Mar12.toPandas()
m12.to_csv('Mar12.csv')
m12.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar13 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-13%'")
m13 = Mar13.toPandas()
m13.to_csv('Mar13.csv')
m13.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar14 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-14%'")
m14 = Mar14.toPandas()
m14.to_csv('Mar14.csv')
m14.describe()

In [ ]:
sqlContext.registerDataFrameAsTable(saveall, "timePlus")
Mar15 = sqlContext.sql("SELECT * FROM timePlus WHERE publicationTime LIKE '%2016-03-15%'")
m15 = Mar15.toPandas()
m15.to_csv('Mar15.csv')
m15.describe()